<a href="https://colab.research.google.com/github/lsuhpchelp/lbrnloniworkshop2019/blob/master/day3_deeplearning_R/dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deep learning with R
===

# Outline
* Install and load R packages
* `keras` package

# Install and load R packages

May take a while on the Colab

R packages to be installed:

In [0]:
install.packages("keras")
install.packages("AmesHousing")
install.packages("rsample")
install.packages("yardstick")

In [0]:
getwd()
list.files()
download.file("https://apache-mxnet.s3-accelerate.dualstack.amazonaws.com/R/data/mnist_csv.zip","mnist_csv.zip")


In [0]:
list.files()

Load R packages:

In [0]:
library(keras)
library(AmesHousing)
library(rsample)
library(yardstick)

# 1. `keras` pakcage
## 1.1 A classification example: `MNIST` data
* MNIST (Mixed National Institute of Standards and Technology
database) is a large database of handwritten digits that is commonly
used for training various image processing systems.
* It consists of images of handwritten digits like these:

> https://drive.google.com/open?id=1DnQzdwWLR4EnWuDteA8H9hgK6JOS3vmI

* `MNIST` data is included in the `keras` package and can be accessed using the `dataset_mnist()` function, which has 60000 training images and 10000 testing images.


In [0]:
mnist <- dataset_mnist()
x_train <- mnist$train$x
y_train <- mnist$train$y
x_test <- mnist$test$x
y_test <- mnist$test$y

* Each image is 28 pixels by 28 pixels. We can interpret this as a big array of numbers.

* We can flatten this array into a vector of 28x28 = 784 numbers. It doesn't matter how we flatten the array, as long as we're consistent between images.
* Then, since the raw data has the grayscale values from integers ranging between 0 to 255, we convert the grayscale values into floating point values ranging between 0 and 1:

In [0]:
# reshape
x_train <- array_reshape(x_train, c(nrow(x_train), 784))
x_test <- array_reshape(x_test, c(nrow(x_test), 784))
# rescale
x_train <- x_train / 255
x_test <- x_test / 255

* For the purposes of this tutorial, we label the y’s as "one hot vectors".
* A one hot vector is a vector which is 0 in most dimensions, and 1 in a single dimension.
*  For example, how to label an “8”?
> * \[0, 0, 0, 0, 0, 0, 0, 1, 0, 0 ]
* The one hot vector can be created using the `to_categorical()` function

In [0]:
y_train <- to_categorical(y_train, 10)
y_test <- to_categorical(y_test, 10)

### 1.1.1 Defining the Model

* A sequential model can be created by the `keras_model_sequential()` function then a series of layer functions.
> * layers are added by using the pipe (`%>%`) operator, each layer is defined by the `layer_dense` function. ("dense" means full-connected) 
> * `units` positive integer, dimensionality of the output space (hidden nodes). The number of hidden nodes in a hidden layer is equal to or less than the number of features in the input but this is not a rule of thumb. 
> * `input_shape` dimensionality of the input (integer) **not** including the samples' axis (samples' axis is 60000 in this example). This argument is required when using this layer as the first layer in a model

In [0]:
model <- keras_model_sequential() 
model %>% 
  layer_dense(units = 256, input_shape = c(784)) %>% # hidden layer 1
  layer_dense(units = 128) %>%                       # hidden layer 2
  layer_dense(units = 10, activation = 'softmax')    # output layer

In [0]:
summary(model)

## Quiz
1. Why the total weights is 235146?
2. Why the hidden nodes in the first hidden layer is set to 256? 

* Next, configure the model with appropriate loss function, optimizer, and metrics.
> * `loss = 'categorical_crossentropy'` remember for classification we use cross-entropy
> *  `optimizer = optimizer_rmsprop()` call an optimizer function 
> * For any classification problem you will want to set this to `metrics = c('accuracy')`

In [0]:
model %>% compile(
  loss = 'categorical_crossentropy',
  optimizer = optimizer_rmsprop(),
  metrics = c('accuracy')
)

### 1.1.2 Model training

In [0]:
history <- model %>% fit(
  x_train, y_train, 
  epochs = 30, batch_size = 128, 
  validation_split = 0.2
)

In [0]:
# one hot encode --> we use model.matrix(...)[, -1] to discard the intercept
data_onehot <- model.matrix(~ ., AmesHousing::make_ames())[, -1] %>% as.data.frame()

# Create training (70%) and test (30%) sets for the AmesHousing::make_ames() data.
# Use set.seed for reproducibility
set.seed(123)
split <- rsample::initial_split(data_onehot, prop = .7, strata = "Sale_Price")
train <- rsample::training(split)
test  <- rsample::testing(split)

# Create & standardize feature sets
# training features
train_x <- train %>% dplyr::select(-Sale_Price)
mean    <- colMeans(train_x)
std     <- apply(train_x, 2, sd)
train_x <- scale(train_x, center = mean, scale = std)

# testing features
test_x <- test %>% dplyr::select(-Sale_Price)
test_x <- scale(test_x, center = mean, scale = std)

# Create & transform response sets
train_y <- log(train$Sale_Price)
test_y  <- log(test$Sale_Price)

# zero variance variables (after one hot encoded) cause NaN so we need to remove
zv <- which(colSums(is.na(train_x)) > 0, useNames = FALSE)
train_x <- train_x[, -zv]
test_x  <- test_x[, -zv]

# What is the dimension of our feature matrix?
dim(train_x)
## [1] 2054  299
dim(test_x)
## [1] 876 299

Layers and nodes

In [0]:
model <- keras_model_sequential() %>%
  layer_dense(units = 10, input_shape = ncol(train_x)) %>%
  layer_dense(units = 5) %>%
  layer_dense(units = 1)

Activation

In [0]:
model <- keras_model_sequential() %>%
  layer_dense(units = 10, activation = "relu", input_shape = ncol(train_x)) %>%
  layer_dense(units = 5, activation = "relu") %>%
  layer_dense(units = 1)

backpropagation

In [0]:
model <- keras_model_sequential() %>%
  
  # network architecture
  layer_dense(units = 10, activation = "relu", input_shape = ncol(train_x)) %>%
  layer_dense(units = 5, activation = "relu") %>%
  layer_dense(units = 1) %>%
  
  # backpropagation
  compile(
    optimizer = "rmsprop",
    loss = "mse",
    metrics = c("mae")
  )

Model training

We’ve created a base model, now we just need to train it with our data. To do so we feed our model into a fit function along with our training data.

In [0]:
learn <- model %>% fit(
  x = train_x,
  y = train_y,
  epochs = 25,
  batch_size = 32,
  validation_split = .2,
  verbose = FALSE
)

In [0]:
learn
## Trained on 1,643 samples, validated on 411 samples (batch_size=32, epochs=25)
## Final epoch (plot to see history):
## val_mean_absolute_error: 1.034
##                val_loss: 4.078
##     mean_absolute_error: 0.4967
##                    loss: 0.5555
plot(learn)

Overfit

In [0]:
model <- keras_model_sequential() %>%
  
  # network architecture
  layer_dense(units = 500, activation = "relu", input_shape = ncol(train_x)) %>%
  layer_dense(units = 250, activation = "relu", input_shape = ncol(train_x)) %>%
  layer_dense(units = 125, activation = "relu") %>%
  layer_dense(units = 1) %>%
  
  # backpropagation
  compile(
    optimizer = "rmsprop",
    loss = "mse",
    metrics = c("mae")
  )

# train our model
learn <- model %>% fit(
  x = train_x,
  y = train_y,
  epochs = 25,
  batch_size = 32,
  validation_split = .2,
  verbose = FALSE
)

plot(learn)


Adjust epochs

Alternatively, if your epochs flatline early then there is no reason to run so many epochs as you are just using extra computational energy with no gain. We can add a callback function inside of fit to help with this.

In [0]:
model <- keras_model_sequential() %>%
  
  # network architecture
  layer_dense(units = 100, activation = "relu", input_shape = ncol(train_x)) %>%
  layer_dense(units = 50, activation = "relu") %>%
  layer_dense(units = 1) %>%
  
  # backpropagation
  compile(
    optimizer = "rmsprop",
    loss = "mse",
    metrics = c("mae")
  )

# train our model
learn <- model %>% fit(
  x = train_x,
  y = train_y,
  epochs = 25,
  batch_size = 32,
  validation_split = .2,
  verbose = FALSE,
  callbacks = list(
    callback_early_stopping(patience = 2)
  )
)

learn
## Trained on 1,643 samples, validated on 411 samples (batch_size=32, epochs=6)
## Final epoch (plot to see history):
## val_mean_absolute_error: 0.8835
##                val_loss: 1.353
##     mean_absolute_error: 0.397
##                    loss: 0.2517

plot(learn)

Add batch normalization


In [0]:
model <- keras_model_sequential() %>%
  
  # network architecture
  layer_dense(units = 100, activation = "relu", input_shape = ncol(train_x)) %>%
  layer_batch_normalization() %>%
  layer_dense(units = 50, activation = "relu") %>%
  layer_batch_normalization() %>%
  layer_dense(units = 1) %>%
  
  # backpropagation
  compile(
    optimizer = "rmsprop",
    loss = "mse",
    metrics = c("mae")
  )

# train our model
learn <- model %>% fit(
  x = train_x,
  y = train_y,
  epochs = 25,
  batch_size = 32,
  validation_split = .2,
  verbose = FALSE,
  callbacks = list(
    callback_early_stopping(patience = 2)
  )
)

learn
## Trained on 1,643 samples, validated on 411 samples (batch_size=32, epochs=25)
## Final epoch (plot to see history):
## val_mean_absolute_error: 0.2016
##                val_loss: 0.07756
##     mean_absolute_error: 0.1786
##                    loss: 0.05482

plot(learn)

Add dropout

In [0]:
model <- keras_model_sequential() %>%
  
  # network architecture
  layer_dense(units = 100, activation = "relu", input_shape = ncol(train_x)) %>%
  layer_batch_normalization() %>%
  layer_dropout(rate = 0.2) %>%
  layer_dense(units = 50, activation = "relu") %>%
  layer_batch_normalization() %>%
  layer_dropout(rate = 0.2) %>%
  layer_dense(units = 1) %>%
  
  # backpropagation
  compile(
    optimizer = "rmsprop",
    loss = "mse",
    metrics = c("mae")
  )

# train our model
learn <- model %>% fit(
  x = train_x,
  y = train_y,
  epochs = 25,
  batch_size = 32,
  validation_split = .2,
  verbose = FALSE,
  callbacks = list(
    callback_early_stopping(patience = 2)
  )
)

learn
## Trained on 1,643 samples, validated on 411 samples (batch_size=32, epochs=19)
## Final epoch (plot to see history):
## val_mean_absolute_error: 0.4557
##                val_loss: 0.4161
##     mean_absolute_error: 1.185
##                    loss: 2.247

plot(learn)

Add weight regularization

In [0]:
model <- keras_model_sequential() %>%
  
  # network architecture
  layer_dense(units = 100, activation = "relu", input_shape = ncol(train_x),
              kernel_regularizer = regularizer_l2(0.001)) %>%
  layer_batch_normalization() %>%
  layer_dense(units = 50, activation = "relu",
              kernel_regularizer = regularizer_l2(0.001)) %>%
  layer_batch_normalization() %>%
  layer_dense(units = 1) %>%
  
  # backpropagation
  compile(
    optimizer = "rmsprop",
    loss = "mse",
    metrics = c("mae")
  )

# train our model
learn <- model %>% fit(
  x = train_x,
  y = train_y,
  epochs = 25,
  batch_size = 32,
  validation_split = .2,
  verbose = FALSE,
  callbacks = list(
    callback_early_stopping(patience = 2)
  )
)

learn
## Trained on 1,643 samples, validated on 411 samples (batch_size=32, epochs=18)
## Final epoch (plot to see history):
## val_mean_absolute_error: 0.5196
##                val_loss: 0.6465
##     mean_absolute_error: 0.342
##                    loss: 0.3858

plot(learn)

Adjust learning rate

In [0]:
model <- keras_model_sequential() %>%
  
  # network architecture
  layer_dense(units = 100, activation = "relu", input_shape = ncol(train_x),
              kernel_regularizer = regularizer_l2(0.001)) %>%
  layer_batch_normalization() %>%
  layer_dense(units = 50, activation = "relu",
              kernel_regularizer = regularizer_l2(0.001)) %>%
  layer_batch_normalization() %>%
  layer_dense(units = 1) %>%
  
  # backpropagation
  compile(
    optimizer = "rmsprop",
    loss = "mse",
    metrics = c("mae")
  )

# train our model
learn <- model %>% fit(
  x = train_x,
  y = train_y,
  epochs = 25,
  batch_size = 32,
  validation_split = .2,
  verbose = FALSE,
  callbacks = list(
    callback_early_stopping(patience = 2),
    callback_reduce_lr_on_plateau()
  )
)

learn
## Trained on 1,643 samples, validated on 411 samples (batch_size=32, epochs=25)
## Final epoch (plot to see history):
## val_mean_absolute_error: 0.2325
##                val_loss: 0.2685
##                      lr: 0.001
##     mean_absolute_error: 0.2119
##                    loss: 0.2482

plot(learn)

Predicting
We can use predict to predict our log sales prices for the test set.

In [0]:
model %>% predict(test_x[1:10,])

In [0]:
(results <- model %>% evaluate(test_x, test_y))

In [0]:
model %>% 
  predict(test_x) %>% 
  broom::tidy() %>% 
  dplyr::mutate(
    truth = test_y, 
    pred_tran = exp(x), 
    truth_tran = exp(truth)
    ) %>%
   yardstick::rmse(truth_tran, pred_tran)

In [0]:
plot(history)

In [0]:
model %>% evaluate(x_test, y_test)

In [0]:
model %>% predict_classes(x_test)